In [ ]:
import asyncio
import logging
import sys

from astropy.time import Time

from lsst.ts import salobj

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
d = salobj.Domain()
CCCamera = salobj.Remote(d, 'CCCamera')
await CCCamera.start_task

In [ ]:
date = Time.now()
group_id = f'CALSET_{date.tai.strftime("%Y%m%d_%H%M")}'
CCCamera.cmd_takeImages.set(numImages=1,
                            expTime=0.,
                            shutter=False,
                            imageType="BIAS",
                            groupId=group_id,
                            science=True,
                            guide=False,
                            wfs=False)
await CCCamera.cmd_takeImages.start(timeout=60)